In [1]:
import time
from helper import get_openai_api_key
from openai import OpenAI
import markdown

class O1ResponseGenerator:
    def __init__(self, o1_model="o1-mini"):
        """
        Initialize the O1ResponseGenerator with a model name.
        """
        self.o1_model = o1_model
        openai_api_key = get_openai_api_key()
        self.client = OpenAI(api_key=openai_api_key)

    def _calculate_cost(self, prompt_tokens, cached_input_tokens, output_tokens):
        """
        Internal method to calculate the total cost based on tokens used.
        """
        model_prices = {
            "o1": {
                "input_token_price": 15.00 / 1_000_000,
                "cached_input_token_price": 7.50 / 1_000_000,
                "output_token_price": 60.00 / 1_000_000,
            },
            "o1-mini": {
                "input_token_price": 3.00 / 1_000_000,
                "cached_input_token_price": 1.50 / 1_000_000,
                "output_token_price": 12.00 / 1_000_000,
            },
        }

        if self.o1_model not in model_prices:
            self.o1_model = "o1"

        prices = model_prices[self.o1_model]
        input_token_cost = (prompt_tokens - cached_input_tokens) * prices["input_token_price"]
        cached_input_token_cost = cached_input_tokens * prices["cached_input_token_price"]
        output_token_cost = output_tokens * prices["output_token_price"]

        return input_token_cost + cached_input_token_cost + output_token_cost

    def _save_response_html(self, file_name, response_time, total_cost, response_content):
        """
        Internal method to generate and save HTML from the response content.
        """
        response_html = markdown.markdown(response_content)
        html_content = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <title>O1 Response</title>
            <script type="text/javascript" async 
                src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
            </script>
        </head>
        <body>
            <div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;">
                <h2>{self.o1_model} Model Response</h2>
                <p>Response time: {response_time:.2f} seconds</p>
                <p>Total Cost: {total_cost:.2f} $</p>
            </div>

            <div style="padding: 10px;">
                {response_html}
            </div>
        </body>
        </html>
        """
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(html_content)

        print(f"Response saved to {file_name}")

    def generate_response(self, prompt, system_prompt, file_name="o1_response_test.html"):
        """
        Public method to generate a response using the O1 model.
        """
        start_time = time.time()

        response = self.client.chat.completions.create(
            model=self.o1_model,
            messages=[
                {
                    "role": "user",
                    "content": f"<System Prompt>{system_prompt}</System Prompt>\\n<USER Prompt>{prompt}</USER Prompt>"
                }
            ]
        )

        response_time = time.time() - start_time
        usage = response.usage

        total_cost = self._calculate_cost(
            usage.prompt_tokens,
            usage.prompt_tokens_details.cached_tokens,
            usage.completion_tokens
        )

        response_content = response.choices[0].message.content

        # Save to HTML
        self._save_response_html(file_name, response_time, total_cost, response_content)

        return response

In [3]:
system_prompt = """
- You are an expert Python programmer and refactoring specialist.
- Before refactoring you need to Identify flaws in logic, inefficiencies, potential bugs and correct those.
- Your task is to refactor the provided Python code step by step, convert the code into object-oriented approach.
- Provide the following for each step:
  - A brief explanation of what is being refactored and why the change is necessary.
  - The original code snippet and the refactored version, presented side by side using Python code blocks.
  - Highlight the improvements in the refactored version, such as enhanced readability, better performance, or bug fixes.
- If additional functionality or changes in design are needed (e.g., converting the code to an object-oriented approach or refactoring for async programming), explain the rationale before implementing.
- Structure the output using HTML elements. So it should include all appropriate HTML elements.
  - blocks for Python code and ensure proper syntax highlighting.
- Ensure the tone is professional, constructive, and educational.
- Keep the layout clean and intuitive, ensuring the user can easily follow the suggestions and apply them effectively.
"""

prompt = '''
import time
from helper import get_openai_api_key
from openai import OpenAI
import markdown

class O1ResponseGenerator:
    def __init__(self, o1_model="o1"):
        """
        Initialize the O1ResponseGenerator with a model name.
        """
        self.o1_model = o1_model
        openai_api_key = get_openai_api_key()
        self.client = OpenAI(api_key=openai_api_key)

    def _calculate_cost(self, prompt_tokens, cached_input_tokens, output_tokens):
        """
        Internal method to calculate the total cost based on tokens used.
        """
        model_prices = {
            "o1": {
                "input_token_price": 15.00 / 1_000_000,
                "cached_input_token_price": 7.50 / 1_000_000,
                "output_token_price": 60.00 / 1_000_000,
            },
            "o1-mini": {
                "input_token_price": 3.00 / 1_000_000,
                "cached_input_token_price": 1.50 / 1_000_000,
                "output_token_price": 12.00 / 1_000_000,
            },
        }

        if self.o1_model not in model_prices:
            self.o1_model = "o1"

        prices = model_prices[self.o1_model]
        input_token_cost = (prompt_tokens - cached_input_tokens) * prices["input_token_price"]
        cached_input_token_cost = cached_input_tokens * prices["cached_input_token_price"]
        output_token_cost = output_tokens * prices["output_token_price"]

        return input_token_cost + cached_input_token_cost + output_token_cost

    def _save_response_html(self, file_name, response_time, total_cost, response_content):
        """
        Internal method to generate and save HTML from the response content.
        """
        response_html = markdown.markdown(response_content)
        html_content = f"""
        <html>
        <head>
            <meta charset="UTF-8">
            <title>O1 Response</title>
            <script type="text/javascript" async 
                src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML">
            </script>
        </head>
        <body>
            <div style="background-color: #f0fff8; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;">
                <h2>{self.o1_model} Model Response</h2>
                <p>Response time: {response_time:.2f} seconds</p>
                <p>Total Cost: {total_cost:.2f} $</p>
            </div>

            <div style="padding: 10px;">
                {response_html}
            </div>
        </body>
        </html>
        """
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(html_content)

        print(f"Response saved to {file_name}")

    def generate_response(self, prompt, system_prompt, file_name="o1_response_test.html", display_output=False):
        """
        Public method to generate a response using the O1 model.
        """
        start_time = time.time()

        response = self.client.chat.completions.create(
            model=self.o1_model,
            messages=[
                {
                    "role": "user",
                    "content": f"<System Prompt>{system_prompt}</System Prompt>\\n<USER Prompt>{prompt}</USER Prompt>"
                }
            ]
        )

        response_time = time.time() - start_time
        usage = response.usage

        total_cost = self._calculate_cost(
            usage.prompt_tokens,
            usage.prompt_tokens_details.cached_tokens,
            usage.completion_tokens
        )

        response_content = response.choices[0].message.content

        if display_output:
            print(f"Model: {self.o1_model}")
            print(f"Response time: {response_time:.2f} seconds")
            print(f"Total Cost: {total_cost:.2f} $\\n")
            print(response_content)

        # Save to HTML
        self._save_response_html(file_name, response_time, total_cost, response_content)

        return response
'''


In [ ]:
response_generator = O1ResponseGenerator(o1_model="o1-mini")
# Generate the response
response = response_generator.generate_response(
    prompt=prompt,
    system_prompt=system_prompt,
    file_name="code_update1.html",
    display_output=False
)

Model: o1-mini
Response time: 47.26 seconds
Total Cost: 0.07 $\n
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Step-by-Step Refactoring of O1ResponseGenerator</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        h2 { color: #2E8B57; }
        .step { margin-bottom: 40px; }
        .explanation { margin-bottom: 10px; }
        .code-container { display: flex; gap: 20px; }
        pre { background-color: #f5f5f5; padding: 10px; border-radius: 5px; overflow-x: auto; width: 48%; }
        .improvements { margin-top: 10px; color: #555555; }
    </style>
</head>
<body>
    <h1>Refactoring of `O1ResponseGenerator` Class</h1>

    <div class="step">
        <h2>Step 1: Optimize Import Statements</h2>
        <div class="explanation">
            <p>Optimizing import statements enhances code readability and ensures that only necessary modules are loaded. Additionally, organizing imports into standard library, third-party, and